In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind

In [2]:
def get_significance_stars(p_value):
    """
    Return significance stars based on the p-value.
    """
    if p_value <= 0.001:
        return '***'
    elif p_value <= 0.01:
        return '**'
    elif p_value <= 0.05:
        return '*'
    else:
        return ''

In [3]:
def add_stat_annotation(ax, x1, x2, y, p_value, significance_stars, col_name):
    """
    Add a line and significance stars between two boxes on the plot.
    """
    line_y = y + 0.05 * y  # Offset for line above the box
    ax.plot([x1, x1, x2, x2], [y, line_y, line_y, y], lw=1.5, color='black')
    ax.text((x1 + x2) * 0.5, line_y, significance_stars, ha='center', va='bottom', color=col_name, fontsize=12)


In [4]:
def plot_tf_distributions(sampled_data, tf_preferences, output_folder, plot_type='boxplot'):
    """
    Plot distributions of distances for TFs, colored by their class, with p-value annotations.

    Parameters:
    - sampled_data: DataFrame containing the distances with TFs and their classes.
    - tf_preferences: DataFrame containing TF preferences and their corresponding classes.
    - output_folder: Path to the folder where the plots will be saved.
    - plot_type: Type of plot to create ('boxplot' or 'violinplot').
    """
    # Rename Class column in tf_preferences for clarity
    tf_preferences = tf_preferences.rename(columns={'Class': 'Class_from_pref'})

    # Merge sampled_data with tf_preferences to get the group (Class) information
    merged_data = sampled_data.merge(tf_preferences[['TF', 'Class_from_pref']], on='TF', how='left')

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Plotting
    plt.figure(figsize=(15, 10))
    ax = plt.gca()

    sns.boxplot(x='TF', y='Distance', hue='Class_from_pref', data=merged_data, palette='Set2', showfliers=False)

    # Perform statistical tests and annotate with p-values
    unique_groups = merged_data['Class_from_pref'].unique()
    box_positions = [0, 7, 10] 
    y_group = merged_data['Distance'].max() / 100
    gap_increment = (y_group / 10)

    current_y = y_group
    for i, g1 in enumerate(unique_groups):
        for j, g2 in enumerate(unique_groups):
            if i < j:  # Compare each pair of TFs only once
                g1_data = merged_data[merged_data['Class_from_pref'] == g1]
                g2_data = merged_data[merged_data['Class_from_pref'] == g2]
                
                g1_distances = g1_data['Distance']
                g2_distances = g2_data['Distance']

                # Perform t-test
                stat, p_value = ttest_ind(g1_distances, g2_distances, equal_var=False)

                # Get significance stars based on p-value
                stars = get_significance_stars(p_value)
                if stars:  # Only annotate significant comparisons
                    add_stat_annotation(ax, box_positions[i], box_positions[j], current_y, p_value, stars, "red")
                    current_y += gap_increment
                
                unique_tfs_g1 = g1_data['TF'].unique()
                unique_tfs_g2 = g2_data['TF'].unique()
                y_max_tf = merged_data['Distance'].median() * 8  # Adjust to be a bit above the max value
                box_positions_g1 = range(len(unique_tfs_g1))  
                box_positions_g2 = range(len(unique_tfs_g1), len(unique_tfs_g1) + len(unique_tfs_g2))
                gap_increment_tf = (y_max_tf * 0.5) 

                # TF-TF wise p-values
                current_y = y_max_tf
                for k, tf1 in enumerate(unique_tfs_g1):
                    for l, tf2 in enumerate(unique_tfs_g1):
                        if k < l:  # Compare each pair of TFs only once
                            tf1_distances = g1_data[g1_data['TF'] == tf1]['Distance']
                            tf2_distances = g1_data[g1_data['TF'] == tf2]['Distance']

                            stat, p_value = ttest_ind(tf1_distances, tf2_distances, equal_var=False)

                            stars = get_significance_stars(p_value)

                            if stars:  
                                # x_pos = (i + j) / 2  # Average position between two TFs
                                # plt.text(x=x_pos, y=y_max, s=stars, ha='center', va='bottom', fontsize=14, color='red')
                                add_stat_annotation(ax, box_positions_g1[k], box_positions_g1[l], current_y, p_value, stars, "black")
                                current_y += gap_increment

                current_y = y_max_tf
                for k, tf1 in enumerate(unique_tfs_g2):
                    for l, tf2 in enumerate(unique_tfs_g2):
                        if k < l:  # Compare each pair of TFs only once
                            tf1_distances = g2_data[g2_data['TF'] == tf1]['Distance']
                            tf2_distances = g2_data[g2_data['TF'] == tf2]['Distance']

                            stat, p_value = ttest_ind(tf1_distances, tf2_distances, equal_var=False)

                            stars = get_significance_stars(p_value)

                            if stars:  # Only annotate significant comparisons
                                add_stat_annotation(ax, box_positions_g2[k], box_positions_g2[l], current_y, p_value, stars, "black")
                                current_y += gap_increment

    plt.title(f'TF-wise Distribution of Distances ({plot_type.capitalize()})')
    ax.set_xlabel('Transcription Factors (TF)', fontsize=16)  # X-axis label font size
    ax.set_ylabel('Distance', fontsize=16)  # Y-axis label font size

    # Adjust tick label font size
    ax.tick_params(axis='x', labelsize=14)  # X-axis tick labels font size
    ax.tick_params(axis='y', labelsize=14)
    plt.xticks(rotation=60, ha='right')

    # Save the figure
    plt.savefig(os.path.join(output_folder, f'TF_wise_distribution_{plot_type}.png'), bbox_inches="tight", dpi=600)
    plt.close()

    print(f'{plot_type.capitalize()} plot saved to {output_folder}')



In [5]:
def plot_tf_distributions_without_sig(sampled_data, tf_preferences, output_folder, plot_type='boxplot', sort_by='mean'):
    """
    Plot distributions of distances for TFs, colored by their class, sorted by mean or max of distributions.

    Parameters:
    - sampled_data: DataFrame containing the distances with TFs and their classes.
    - tf_preferences: DataFrame containing TF preferences and their corresponding classes.
    - output_folder: Path to the folder where the plots will be saved.
    - plot_type: Type of plot to create ('boxplot' or 'violinplot').
    - sort_by: Statistic to sort by ('mean' or 'max').
    """
    # Rename Class column in tf_preferences for clarity
    tf_preferences = tf_preferences.rename(columns={'Class': 'Class_from_pref'})

    # Merge sampled_data with tf_preferences to get the group (Class) information
    merged_data = sampled_data.merge(tf_preferences[['TF', 'Class_from_pref']], on='TF', how='left')

    # Calculate the mean or max distance per TF for sorting
    if sort_by == 'mean':
        tf_order = merged_data.groupby('TF')['Distance'].mean().sort_values().index
    elif sort_by == 'max':
        tf_order = merged_data.groupby('TF')['Distance'].max().sort_values().index
    else:
        raise ValueError("sort_by parameter should be either 'mean' or 'max'")

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Plotting
    plt.figure(figsize=(15, 10))
    ax = plt.gca()

    sns.boxplot(x='TF', y='Distance', hue='Class_from_pref', data=merged_data, 
                palette='Set2', showfliers=False, order=tf_order)

    plt.title(f'TF-wise Distribution of Distances (Sorted by {sort_by.capitalize()})')
    ax.set_xlabel('Transcription Factors (TF)', fontsize=16)  # X-axis label font size
    ax.set_ylabel('Distance', fontsize=16)  # Y-axis label font size

    # Adjust tick label font size
    ax.tick_params(axis='x', labelsize=14)  # X-axis tick labels font size
    ax.tick_params(axis='y', labelsize=14)
    plt.xticks(rotation=60, ha='right')

    # Save the figure
    plt.savefig(os.path.join(output_folder, f'TF_wise_distribution_sorted_by_{sort_by}_{plot_type}.png'), 
                bbox_inches="tight", dpi=600)
    plt.close()

    print(f'{plot_type.capitalize()} plot saved to {output_folder}')

# Example usage
# plot_tf_distributions_without_sig(sampled_data, tf_preferences, 'output_folder_path', plot_type='boxplot', sort_by='mean')


In [6]:
def sample_distances(distance_file):
    # Read only the 4th and 8th columns from the file
    distances = pd.read_csv(distance_file, sep='\t', usecols=[3, 7], header=None, names=['TF', 'Distance'])
    
    return distances


In [7]:
def write_tf_statistics(sampled_data, output_folder, tf, n_instances):
    """
    Write statistics of distances for a TF to a text file.
    
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Calculate statistics for each class
    stats = sampled_data[sampled_data['TF'] == tf,'Distance'].describe()

    # Rename columns for clarity
    stats.rename(columns={
        'count': 'Count',
        'mean': 'Mean',
        'std': 'StdDev',
        'min': 'Min',
        '25%': 'Q1',
        '50%': 'Median',
        '75%': 'Q3',
        'max': 'Max'
    }, inplace=True)

    # Define output file name based on distance type
    output_file = os.path.join(output_folder, f'{tf}_statistics.txt')

    # Write the statistics to a text file
    with open(output_file, 'w') as f:
        f.write(stats.to_string(index=False))

    print(f'Statistics for {tf} distances written to {output_file}')

In [14]:
output_folder = '/home/manisha/Desktop/shape_clusters/hepg2/raw_bed/distances/domains/' 
n = 10000  
tf_preference_file = '/home/manisha/Desktop/shape_clusters/hepg2/raw_bed/distances/domains/domain_dist/tf_preference_subset' 
first_distances_pattern = '{tf}_distances.txt' 

plot_output_folder = '/home/manisha/Desktop/shape_clusters/hepg2/raw_bed/distances/domains/domain_dist/'
os.makedirs(plot_output_folder, exist_ok=True)


In [15]:
tf_preferences = pd.read_csv(tf_preference_file, sep='\t', header=None, names=['TF', 'Class'])

In [16]:
all_sampled_data_first = []
summary_data = []
# Loop through each TF and sample from their specific distance files
for tf in tf_preferences['TF']:
    print(f"Processing {tf}")

    first_distance_file = os.path.join(output_folder, first_distances_pattern.format(tf=tf.strip()))
    sampled_first = sample_distances(first_distance_file)
    sampled_first['TF'] = tf
    sampled_first['Class'] = tf_preferences.loc[tf_preferences['TF'] == tf, 'Class'].values[0]

    # print(sampled_first.head())

    # Generate summary statistics
    stats = sampled_first['Distance'].describe()

    # Append the TF name, class, and stats to the summary data
    summary_data.append([tf, sampled_first['Class'].iloc[0], *stats.values])
    
    summary_columns = ['TF', 'Class', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
    summary_df = pd.DataFrame(summary_data, columns=summary_columns)
    
    all_sampled_data_first.append(sampled_first)

Processing ATF2
Processing CEBPB
Processing CTCF
Processing IKZF1
Processing RAD21
Processing YBX1
Processing ZNF207
Processing NR2C2


In [17]:
print(summary_df)


       TF     Class    count           mean           std  min       25%  \
0    ATF2     Shape  49078.0  209405.287583  3.711365e+05 -1.0  32333.00   
1   CEBPB     Shape  57578.0  116238.045000  2.216520e+05  0.0  18350.00   
2    CTCF     Shape  67504.0   88274.892066  1.933372e+05  0.0   1634.75   
3   IKZF1     Shape   1781.0  261328.576642  1.195111e+06  0.0  21537.00   
4   RAD21     Shape  52102.0   76841.996526  1.610973e+05 -1.0      0.00   
5    YBX1     Shape   2832.0  187924.974576  8.930782e+05  0.0  20159.50   
6  ZNF207     Shape   1788.0  171330.942394  9.613023e+05 -1.0  12302.75   
7   NR2C2  Sequence   1750.0  111000.818857  2.396941e+05  0.0  17676.00   

       50%        75%         max  
0  92042.5  226948.00   8997878.0  
1  53779.0  125881.75   6596346.0  
2  35247.5  100108.50   9126064.0  
3  58018.0  159417.00  35485564.0  
4  28632.0   88699.00   5634155.0  
5  56598.5  145640.25  35485555.0  
6  43917.0  118940.50  35507196.0  
7  51292.5  125578.00   425

In [22]:
sampled_data_first = pd.concat(all_sampled_data_first)


In [23]:
plot_tf_distributions(sampled_data_first, tf_preferences, plot_output_folder )


Boxplot plot saved to /home/manisha/Desktop/shape_clusters/hepg2/raw_bed/distances/domains/domain_dist/


In [20]:
plot_tf_distributions_without_sig(sampled_data_first, tf_preferences, plot_output_folder , sort_by = "mean")


Boxplot plot saved to /home/manisha/Desktop/shape_clusters/hepg2/raw_bed/distances/domains/domain_dist/


In [24]:
output_file = os.path.join(plot_output_folder, f'{n}_distance_sampled.txt')

# Write the statistics to a text file
with open(output_file, 'w') as f:
    f.write(sampled_data_first.to_string(index=False))
